# Using the Equities India (BSE) data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/EquitiesIndiaBse.ipynb)


In [6]:
library(DBI)
library(plutoDbR)
library(plutoR)
library(tidyverse)

options("scipen" = 99999)
options(tibble.width = Inf)

source("/usr/share/pluto/config.R")

#initialize
equitiesIndiaBse <- EquitiesIndiaBse()
symbol <- 'SBIN'

## get BSE's code for a symbol

In [3]:
secInfo <- equitiesIndiaBse$Tickers() %>%
           filter(SYMBOL == symbol) %>%
           collect()

secInfo %>% print()

code <- secInfo$CODE[[1]]


# A tibble: 1 x 7
  ISIN           CODE SYMBOL SERIES NAME                 FACE INDUSTRY
* <chr>         <int> <chr>  <chr>  <chr>               <int> <chr>   
1 INE062A01020 500112 SBIN   A      State Bank Of India     1 Banks   


### fetch some "misc" info

In [4]:
maxDt <- (equitiesIndiaBse$MiscInfo() %>%
  summarize(MAX_DT = max(TIME_STAMP)) %>%
  collect())$MAX_DT[[1]]

print("latest misc info: ")
equitiesIndiaBse$MiscInfo() %>%
  filter(TIME_STAMP == maxDt & CODE == code) %>%
  print(n=Inf)

[1] "latest misc info: "
# Source:   lazy query [?? x 5]
# Database: NORWAY:StockViz:R
    CODE TIME_STAMP FF_MKT_CAP_CR FULL_MKT_CAP_CR D2T_PCT
   <int> <chr>              <dbl>           <dbl>   <dbl>
1 500112 2019-09-27       107932.         251005.    13.7


### fetch the latest end-of-day prices

In [7]:
maxDt <- (equitiesIndiaBse$EodTimeSeries() %>%
  summarize(MAX_DT = max(TIME_STAMP)) %>%
  collect())$MAX_DT[[1]]

print("latest end-of-day prices:")
equitiesIndiaBse$EodTimeSeries() %>%
  filter(TIME_STAMP == maxDt & CODE == code) %>%
  print()

[1] "latest end-of-day prices:"
# Source:   lazy query [?? x 13]
# Database: NORWAY:StockViz:R
    CODE NAME       SERIES TYPE  TIME_STAMP  HIGH   LOW  OPEN CLOSE  LAST
   <int> <chr>      <chr>  <chr> <chr>      <dbl> <int> <int> <dbl> <dbl>
1 500112 STATE BANK A      Q     2019-09-27  286.   280   283  281.  281.
   VOLUME NUM_TRADES  TURNOVER
    <int>      <int>     <int>
1 1793222      12164 507061773


### fetch the last 10 day EOD prices

In [8]:
print("last 10 day EOD prices:")
equitiesIndiaBse$EodTimeSeries() %>%
  filter(CODE == code) %>%
  arrange(desc(TIME_STAMP)) %>%
  head(10) %>%
  print(n=Inf)

[1] "last 10 day EOD prices:"
# Source:     lazy query [?? x 13]
# Database:   NORWAY:StockViz:R
# Ordered by: desc(TIME_STAMP)
     CODE NAME       SERIES TYPE  TIME_STAMP  HIGH   LOW  OPEN CLOSE  LAST
    <int> <chr>      <chr>  <chr> <chr>      <dbl> <dbl> <dbl> <dbl> <dbl>
 1 500112 STATE BANK A      Q     2019-09-27  286.  280   283   281.  281.
 2 500112 STATE BANK A      Q     2019-09-26  290.  277   282.  282.  282.
 3 500112 STATE BANK A      Q     2019-09-25  299.  279.  299.  280.  280.
 4 500112 STATE BANK A      Q     2019-09-24  315   301   313.  302.  302.
 5 500112 STATE BANK A      Q     2019-09-23  316.  300.  308.  314.  314.
 6 500112 STATE BANK A      Q     2019-09-20  308.  269.  274.  302.  302.
 7 500112 STATE BANK A      Q     2019-09-19  280.  273.  280.  274.  274.
 8 500112 STATE BANK A      Q     2019-09-18  283.  274.  277   280.  280.
 9 500112 STATE BANK A      Q     2019-09-17  286.  273.  285.  274.  274.
10 500112 STATE BANK A      Q     2019-09-16  2

### fetch the last 24 quarter EPS

In [9]:
refs <- equitiesIndiaBse$CorporateResults() %>%
  filter(CODE == code & KEY %like% '%diluted%' & KEY %like% '%after%') %>%
  arrange(desc(PERIOD_END)) %>%
  collect() %>%
  mutate(PERIOD_END = as.Date(PERIOD_END), PERIOD_BEGIN = as.Date(PERIOD_BEGIN)) %>%
  mutate(P_DIFF = PERIOD_END-PERIOD_BEGIN) %>%
  filter(P_DIFF < 100) %>%
  head(24)

print("the last 24 quarter EPS for SBIN: ")
refs %>% print(n=Inf)

[1] "the last 24 quarter EPS for SBIN: "
# A tibble: 24 x 8
     CODE PERIOD_BEGIN PERIOD_END IS_AUDITED
    <int> <date>       <date>     <lgl>     
 1 500112 2019-04-01   2019-06-30 FALSE     
 2 500112 2019-01-01   2019-03-31 TRUE      
 3 500112 2018-10-01   2018-12-31 FALSE     
 4 500112 2018-07-01   2018-09-30 FALSE     
 5 500112 2018-04-01   2018-06-30 FALSE     
 6 500112 2018-01-01   2018-03-31 TRUE      
 7 500112 2017-10-01   2017-12-31 FALSE     
 8 500112 2017-07-01   2017-09-30 FALSE     
 9 500112 2017-04-01   2017-06-30 FALSE     
10 500112 2017-01-01   2017-03-31 TRUE      
11 500112 2016-10-01   2016-12-31 FALSE     
12 500112 2016-07-01   2016-09-30 FALSE     
13 500112 2016-04-01   2016-06-30 FALSE     
14 500112 2015-10-01   2015-12-31 FALSE     
15 500112 2015-07-01   2015-09-30 FALSE     
16 500112 2015-04-01   2015-06-30 FALSE     
17 500112 2014-10-01   2014-12-31 FALSE     
18 500112 2014-07-01   2014-09-30 FALSE     
19 500112 2014-04-01   2014-06-30 FALSE 

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)